In [ ]:
# notebooks/1_build_vector_store.ipynb
import pandas as pd
import faiss
import numpy as np
import pickle
import sys
from pathlib import Path

# Añadir el directorio raíz del proyecto al path para importar módulos propios
# Esto asume que ejecutas el notebook desde la raíz del proyecto
project_root = Path.cwd() # O especifica la ruta si es diferente
sys.path.append(str(project_root))
print(f"Project root added to path: {project_root}")

# Importar funciones de embeddings y configuración
# Asegúrate de que config.py y embeddings.py funcionen correctamente antes
try:
    from app.core.embeddings import get_embeddings, get_embedding_model
    from app.core.config import settings
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Please ensure you are running this notebook from the project root directory")
    print("and that the app structure and __init__.py files are correct.")
    sys.exit(1) # Salir si no se pueden importar

# --- 1. Cargar Datos ---
data_path = project_root / "data/raw_data.csv"
print(f"Loading data from: {data_path}")
try:
    df = pd.read_csv(data_path)
    # Asegúrate de que la columna con el texto principal se llame 'Texto' o ajústalo
    if 'Texto' not in df.columns:
        raise ValueError("CSV must contain a 'Texto' column.")
    print(f"Data loaded successfully. Shape: {df.shape}")
    print(df.head())
except FileNotFoundError:
    print(f"Error: Data file not found at {data_path}")
    sys.exit(1)
except Exception as e:
    print(f"Error loading data: {e}")
    sys.exit(1)

# --- 2. Generar Embeddings ---
texts_to_embed = df['Texto'].tolist()
print(f"Generating embeddings for {len(texts_to_embed)} documents...")
# Usamos la función get_embeddings que maneja la carga del modelo
embeddings = get_embeddings(texts_to_embed)
embeddings_np = np.array(embeddings).astype('float32') # FAISS requiere numpy float32
print(f"Embeddings generated. Shape: {embeddings_np.shape}")

# --- 3. Construir Índice FAISS ---
dimension = embeddings_np.shape[1] # Dimensión de los embeddings
# Usaremos un índice simple FlatL2 para empezar (búsqueda exacta)
# Para datasets grandes, considera IndexIVFFlat o IndexHNSWFlat
index = faiss.IndexFlatL2(dimension)
# index = faiss.IndexIDMap(index) # Opcional: para mapear IDs de FAISS a IDs originales

print("Adding embeddings to FAISS index...")
# Si usas IndexIDMap:
# ids = df['ID'].values.astype('int64') # Asegúrate de que los IDs son int64
# index.add_with_ids(embeddings_np, ids)
# Si NO usas IndexIDMap (los IDs serán las posiciones 0, 1, 2...):
index.add(embeddings_np)

print(f"Index built. Total vectors in index: {index.ntotal}")

# --- 4. Guardar Índice y Metadatos ---
# Crear directorio si no existe
settings.faiss_index_path.parent.mkdir(parents=True, exist_ok=True)

print(f"Saving FAISS index to: {settings.faiss_index_path}")
faiss.write_index(index, str(settings.faiss_index_path))

# Guardar metadatos (ej: mapeo de índice FAISS a texto original o ID del CSV)
# Si NO usaste IndexIDMap, el ID de FAISS es la posición en la lista original
metadata = {i: {'text': text, 'original_id': df.iloc[i]['ID']}
            for i, text in enumerate(texts_to_embed)}
# Si SÍ usaste IndexIDMap, el ID ya está en el índice, podrías guardar solo texto vs ID original
# metadata = df.set_index('ID')['Texto'].to_dict() # Ejemplo

print(f"Saving metadata to: {settings.faiss_metadata_path}")
with open(settings.faiss_metadata_path, 'wb') as f:
    pickle.dump(metadata, f)

print("Index and metadata saved successfully.")

# --- 5. Prueba Rápida (Opcional) ---
print("\n--- Quick Test ---")
try:
    # Cargar modelo solo para query (o reutilizar)
    query_text = "Información sobre ventas del producto A"
    query_embedding = get_embedding_model().embed_query(query_text)
    query_embedding_np = np.array([query_embedding]).astype('float32')

    k = 3 # Número de vecinos más cercanos a buscar
    print(f"Searching for top {k} similar documents for query: '{query_text}'")
    distances, indices = index.search(query_embedding_np, k)

    print("Results:")
    for i in range(k):
        faiss_id = indices[0][i]
        distance = distances[0][i]
        # Recuperar metadatos usando el índice FAISS devuelto
        doc_info = metadata.get(faiss_id, {"text": "Metadata not found"})
        print(f"  - Rank {i+1}: FAISS ID {faiss_id}, Distance: {distance:.4f}")
        print(f"    Text: {doc_info.get('text', 'N/A')}")
        print(f"    Original ID: {doc_info.get('original_id', 'N/A')}")

except Exception as e:
    print(f"Error during quick test: {e}")